# Libraries

In [1]:
#GET request from the API
import requests
#save acess tokens and file management for the database
import os
#json responses from the API
import json
#data treatment
import pandas as pd
#save the data in csv format
import csv

from dotenv import load_dotenv

load_dotenv()

#export BEARER_TOKEN='AAAAAAAAAAAAAAAAAAAAAAVTfAEAAAAA0oSU10BYW76uhA6HBBhPRsAie78%3DviUudqG4tDSkO0INYyO3yYtdmqmoAqsoC1L8W6ICJuuIFtrooq'

True

In [2]:
#Create the auth() function that will contain the Bearer Token
bearer_token = os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
#query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


In [3]:
def create_url():
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    usernames = "usernames=elonmusk,GiancarloChaux,cafecomferri,tcinvestimentos,MooncatzNFT,Cristiano,cz_binance,VitalikButerin,BuzzFeed,TheEconomist,darth,KindaFunnyVids,GuyKawasaki,VelocidadeDepre,GordonRamsay,guardian,chrissyteigen,kalesalad,nytimes,mental_floss,TEDTalks,cnnbrk,brainpicker,JenniferPreston,atlasobscura,britneyspears,BillGates,Twitter,JeffBezos,jtimberlake,SpaceX,selenagomez,KimKardashian,narendramodi,YouTube,TheEllenShow,ladygaga,taylorswift13,rihanna,katyperry,justinbieber,BarackObama,NASA,Oprah,FallonTonight,jimmyfallon,Sethrogen,JimCarrey,SamuelLJackson,DannyDeVito"
    user_fields = "user.fields=id,description,name,public_metrics,created_at"
    #ids = "ids=1278747501642657792,1255542774432063488"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    #url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)

    return url

In [4]:
def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


In [5]:
def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))
#def main():
#    json_response = connect_to_endpoint(search_url, query_params)
#    print(json.dumps(json_response, indent=4, sort_keys=True))
    with open('data.json', 'w') as f:
        json.dump(json_response, f)

    df = pd.DataFrame(json_response['data'])
    df.to_csv('data.csv')

if __name__ == "__main__":
    main()


200
{
    "data": [
        {
            "created_at": "2009-06-02T20:12:29.000Z",
            "description": "Mars & Cars, Chips & Dips",
            "id": "44196397",
            "name": "Elon Musk",
            "public_metrics": {
                "followers_count": 102081612,
                "following_count": 119,
                "listed_count": 97251,
                "tweet_count": 18791
            },
            "username": "elonmusk"
        },
        {
            "created_at": "2015-08-20T15:11:26.000Z",
            "description": "follow me for tweets on NFTs and web 3 | discord https://t.co/WK3S6hD21i",
            "id": "3432951370",
            "name": "Giancarlo",
            "public_metrics": {
                "followers_count": 84338,
                "following_count": 999,
                "listed_count": 784,
                "tweet_count": 2459
            },
            "username": "GiancarloChaux"
        },
        {
            "created_at": "2011-12-15T21:46:17

In [14]:
df = pd.read_csv('data.csv')
df.tail()
df2 = pd.DataFrame()

In [15]:
df2[['followers_count','following_count', 'tweet_count','listed_count']] = df["public_metrics"].str.split(',', expand=True)
df2.tail()


,followers_count,following_count,tweet_count,listed_count
45,{'followers_count': 51334622,'following_count': 11020,'tweet_count': 14161,'listed_count': 61757}
46,{'followers_count': 9349200,'following_count': 1838,'tweet_count': 11495,'listed_count': 11021}
47,{'followers_count': 19076018,'following_count': 1,'tweet_count': 4698,'listed_count': 53087}
48,{'followers_count': 9041946,'following_count': 862,'tweet_count': 3850,'listed_count': 17470}
49,{'followers_count': 4388364,'following_count': 25,'tweet_count': 7441,'listed_count': 20206}


In [20]:
df3 = pd.DataFrame()
df3[['string','followers_count']] = df2["followers_count"].str.split(':', expand=True)
df3.drop('string', axis=1, inplace=True)
df3[['string1','following_count']] = df2["following_count"].str.split(':', expand=True)
df3.drop('string1', axis=1, inplace=True)
df3[['string2','tweet_count']] = df2["tweet_count"].str.split(':', expand=True)
df3.drop('string2', axis=1, inplace=True)
df3[['string3','listed_count']] = df2["listed_count"].str.split(':', expand=True)
df3.drop('string3', axis=1, inplace=True)

df3["listed_count"].str.replace("}","", regex=True)
#df4['listed_count'] = df3['string4']
#df4.drop('string4', axis=1, inplace=True)


df3.tail()

,followers_count,following_count,tweet_count,listed_count
45,51334622,11020,14161,61757}
46,9349200,1838,11495,11021}
47,19076018,1,4698,53087}
48,9041946,862,3850,17470}
49,4388364,25,7441,20206}


In [24]:
df = df.join(df3, lsuffix='_caller', rsuffix='_other')


In [25]:
df.tail()

,Unnamed: 0,description,username,public_metrics,id,name,created_at,followers_count_caller,following_count_caller,tweet_count_caller,listed_count_caller,followers_count_other,following_count_other,tweet_count_other,listed_count_other
45,45,astrophysicist,jimmyfallon,"{'followers_count': 51334622, 'following_count...",15485441,Fallon.eth,2008-07-18T19:46:50.000Z,51334622,11020,14161,61757},51334622,11020,14161,61757}
46,46,I... Tweet!!!!!!,Sethrogen,"{'followers_count': 9349200, 'following_count'...",443215941,Seth Rogen,2011-12-21T23:17:52.000Z,9349200,1838,11495,11021},9349200,1838,11495,11021}
47,47,The ONLY official social media account for Act...,JimCarrey,"{'followers_count': 19076018, 'following_count...",52551600,Jim Carrey,2009-06-30T22:58:44.000Z,19076018,1,4698,53087},19076018,1,4698,53087}
48,48,NaN,SamuelLJackson,"{'followers_count': 9041946, 'following_count'...",75974281,Samuel L. Jackson,2009-09-21T06:47:04.000Z,9041946,862,3850,17470},9041946,862,3850,17470}
49,49,"I'm an actor, director and producer.",DannyDeVito,"{'followers_count': 4388364, 'following_count'...",71876190,Danny DeVito,2009-09-05T19:44:48.000Z,4388364,25,7441,20206},4388364,25,7441,20206}


In [26]:
df = df.to_csv('treated_data.csv')